In [1]:
#import libraries
import import_ipynb

#import ipynb files
from Extract_Twitter_Trends import Twitter_Trends

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from Extract_Twitter_Trends.ipynb


In [2]:
trends = Twitter_Trends('netherlands', '2023-05-11', 14, 3)
top_trends = trends.get_top_trends()
print (top_trends)

https://getdaytrends.com/netherlands/2023-05-11/14/
{1: '#Imran_Khan', 2: 'Pakistán', 3: 'Gideon'}
